<a href="https://colab.research.google.com/github/ramapriyakp/Portfolio/blob/master/NLP/text_classification_Reuter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reuters-21578 collection
Reuters-21578 is arguably the most commonly used collection for text classification during the last two decades. It has been used in some of the most influential papers on the field.

This dataset contains structured information about newswire articles that can be assigned to several classes, therefore making this a multi-label problem. It has a highly skewed distribution of documents over categories, where a large proportion of documents belong to few topics.

The collection originally consists of 21,578 documents, including documents without topics and typographical errors. For this reason, a subset and split of the collection is traditionally used. This split also focus only on the categories that have at least one document in the training set and the test set. The dataset has 90 categories with a training set of 7769 documents and a test set of 3019 documents.

## Data Collection Stats
Understanding your collection is always the first step of any data science problem. Lets have a quick look at the Reuters collection and its documents.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/NLP

/content/drive/My Drive/NLP


In [0]:
import nltk
nltk.data.path.append('/content/drive/My Drive/NLP')
nltk.download('stopwords')
nltk.download('reuters')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [0]:
from nltk.corpus import reuters

# List of document ids
documents = reuters.fileids()
print("Documents: {}".format(len(documents)))

# Train documents
train_docs_id = list(filter(lambda doc: doc.startswith("train"), documents))
print("Total train documents: {}".format(len(train_docs_id)))

# Test documents
test_docs_id = list(filter(lambda doc: doc.startswith("test"), documents))
print("Total test documents: {}".format(len(test_docs_id)))

Documents: 10788
Total train documents: 7769
Total test documents: 3019


In [0]:
# Example of a document (with multiple labels)
doc = 'training/9865'
print(reuters.raw(doc))
print()

print(reuters.categories(doc))

FRENCH FREE MARKET CEREAL EXPORT BIDS DETAILED
  French operators have requested licences
  to export 675,500 tonnes of maize, 245,000 tonnes of barley,
  22,000 tonnes of soft bread wheat and 20,000 tonnes of feed
  wheat at today's European Community tender, traders said.
      Rebates requested ranged from 127.75 to 132.50 European
  Currency Units a tonne for maize, 136.00 to 141.00 Ecus a tonne
  for barley and 134.25 to 141.81 Ecus for bread wheat, while
  rebates requested for feed wheat were 137.65 Ecus, they said.
  



['barley', 'corn', 'grain', 'wheat']


In [0]:
from operator import itemgetter
from pprint import pprint

# List of categories 
categories = reuters.categories();
print("Number of categories: {}".format(len(categories)))
print()

print(categories)
print()

# Documents per category.
category_distribution = [(category, len(reuters.fileids(category))) 
                         for category in categories]

category_distribution = sorted(category_distribution, 
                               key=itemgetter(1), 
                               reverse=True)

print("Most common categories")
pprint(category_distribution[:5])
print()

print("Least common categories")
pprint(category_distribution[-5:])
print()

Number of categories: 90

['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']

Most common categories
[('earn', 3964),
 ('acq', 2369),
 ('mone

In [0]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

stop_words = stopwords.words("english")

train_docs_id = list(filter(lambda doc: doc.startswith("train"), documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"), documents))

train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]

# Tokenisation 
vectorizer = TfidfVectorizer(stop_words=stop_words)

# Learn and transform train documents
vectorised_train_documents = vectorizer.fit_transform(train_docs)
vectorised_test_documents = vectorizer.transform(test_docs)

# Transform multilabel labels
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([reuters.categories(doc_id) 
                                  for doc_id in train_docs_id])
test_labels = mlb.transform([reuters.categories(doc_id) 
                             for doc_id in test_docs_id])

# Classifier 
classifier = OneVsRestClassifier(LinearSVC(random_state=42))
classifier.fit(vectorised_train_documents, train_labels)
predictions = classifier.predict(vectorised_test_documents)

print("Number of labels assigned: {}".format(sum([sum(prediction) 
                                                  for prediction in predictions])))

Number of labels assigned: 3126


# How well have we done?

**Confusion Matrix**
![Confusion Matrix](https://hackernoon.com/hn-images/1*y4HwoAEgx1Js19hCkPM7XA.png)

__Precision__ — Also called Positive predictive value
The ratio of correct positive predictions to the total predicted positives


![Precision](https://hackernoon.com/hn-images/1*7pXTS_beJfTlkntFHiBh7g.png)

__Recall__ — Also called Sensitivity, Probability of Detection, True Positive Rate

Ratio of correct positive predictions to the total positives examples


![Recall](https://hackernoon.com/hn-images/1*VWD0-yDiy5gBZy7ZTIZgXg.png)

__F Score__: This is a weighted average of the true positive rate (recall) and precision.

In [0]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Show our quality
precision = precision_score(test_labels, predictions, average='micro')
recall = recall_score(test_labels, predictions, average='micro')
f1 = f1_score(test_labels, predictions, average='micro')
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, 
                                                                     recall, 
                                                                     f1))

precision = precision_score(test_labels, predictions, average='macro')
recall = recall_score(test_labels, predictions, average='macro')
f1 = f1_score(test_labels, predictions, average='macro')
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, 
                                                                     recall, 
                                                                     f1))

Micro-average quality numbers
Precision: 0.9517, Recall: 0.7946, F1-measure: 0.8661
Macro-average quality numbers
Precision: 0.6305, Recall: 0.3715, F1-measure: 0.4451


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# If you see a weird warning message about "Precision being ill-defined":
Some metrics can be in a position of indeterminate value when for instance, the classifier decides to not classify any articles in a specific category. This would imply a 0/0 precision. Scikit learn warns us about this, and reports back this quality as 0.0.